In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from model import neural_model as nm
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('final_extracted_a0f66459.csv')
electrode_col_names = [col for col in df.columns if 'GRID' in col ]
X = df[electrode_col_names]
Y = df['mvmt']
Y = (Y=='r_arm_1').astype(int)
num_samples = len(Y)


train_X = X[:int(num_samples*.8)]
test_X = X[int(num_samples*.8):int(num_samples*.9)]
val_X = X[int(num_samples*.9):]
train_Y = Y[:int(num_samples*.8)]
test_Y = Y[int(num_samples*.8):int(num_samples*.9)]
val_Y = Y[int(num_samples*.9):]

In [4]:
class ecog_dataset(Dataset):
    def __init__(self, x, y):
        self.data = x
        self.keys = y
        
    def __len__(self):
        return len(self.keys)
    
    def __getitem__(self, index):
        return torch.tensor(self.data.iloc[index]), torch.tensor([self.keys.iloc[index]])

In [5]:
train_X, test_X, train_Y, test_Y = train_test_split(X, Y,stratify=Y, test_size=0.30)
test_X, val_X, test_Y, val_Y = train_test_split(test_X, test_Y, stratify=test_Y, test_size=1.0/3.0)

trainset = ecog_dataset(train_X, train_Y)
testset = ecog_dataset(test_X, test_Y)
valset = ecog_dataset(val_X, val_Y)

trainloader = DataLoader(trainset, batch_size=100, shuffle=True) #dataset to train on (80% data)
testloader = DataLoader(testset, batch_size=100, shuffle=True) #dataset to test each epoch on (10% data)
validloader = DataLoader(valset, batch_size=100, shuffle=True) #dataset to validate perf on (10% data)

In [6]:
#define network and params
net = nm(64)

loss_func = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(net.parameters(), lr=.001)
best_loss = 99999
best_net = net

In [18]:
import copy
#train
for epoch in range(60):
    running_loss = 0.0
    train_count = 0
    #net = copy.deepcopy(best_net)
    for i, data in enumerate(trainloader, 0):
        net.train()
        inputs, labels = data
        #if(labels.item() == 1):
        optimizer.zero_grad()
        outputs = net(inputs.float())
        #print(outputs)
        loss = loss_func(outputs.float(), labels.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        train_count += 1
    #do epoch test on separate test set
    test_loss = 0.0
    test_count = 0
    for i, data in enumerate(testloader, 0):
        inputs, labels = data
        outputs = net(inputs.float())
        loss = loss_func(torch.round(outputs), labels.float())
        test_loss += loss.item()
        test_count += 1
    if test_loss/test_count < best_loss:
        best_loss = test_loss/test_count
        #best_net = copy.deepcopy(net)
        print('new best model!')
    print("Epoch: ", epoch, 'train loss: ', running_loss/train_count, 
          'test loss: ', test_loss/test_count)

Epoch:  0 train loss:  0.6210436969995499 test loss:  0.6322360038757324
Epoch:  1 train loss:  0.6130714863538742 test loss:  0.6505784839391708
Epoch:  2 train loss:  0.6224280397097269 test loss:  0.6470284312963486
Epoch:  3 train loss:  0.6286038011312485 test loss:  0.6215077787637711
Epoch:  4 train loss:  0.6279243926207224 test loss:  0.6793808043003082
Epoch:  5 train loss:  0.6233350535233816 test loss:  0.6353797167539597
Epoch:  6 train loss:  0.6172137210766474 test loss:  0.6344038248062134
Epoch:  7 train loss:  0.6144675016403198 test loss:  0.6530693471431732
Epoch:  8 train loss:  0.6112454434235891 test loss:  0.6525371074676514
Epoch:  9 train loss:  0.6159925311803818 test loss:  0.640646830201149
Epoch:  10 train loss:  0.6172730525334676 test loss:  0.6205847263336182
Epoch:  11 train loss:  0.6133063832918803 test loss:  0.6486046612262726
Epoch:  12 train loss:  0.6138452291488647 test loss:  0.6478706002235413
Epoch:  13 train loss:  0.6200531125068665 test l

In [20]:
from model import neural_model as nm
import torch
best_net = nm(64)
best_net.load_state_dict(torch.load('curr_best.pyt'))
print(torch.load('curr_best.pyt'))
print(type)

OrderedDict([('fc1.weight', tensor([[ 0.1241, -0.0843, -0.1125,  ..., -0.0874, -0.1034, -0.1154],
        [ 0.0672,  0.1450,  0.0301,  ...,  0.0862,  0.0861,  0.0212],
        [-0.1922,  0.0761, -0.1744,  ...,  0.0087, -0.0748, -0.0502],
        ...,
        [-0.2329, -0.1717, -0.1205,  ..., -0.0254,  0.0066, -0.1323],
        [ 0.1443,  0.0582, -0.0565,  ...,  0.1013,  0.0073,  0.0324],
        [-0.1584, -0.2285,  0.0037,  ..., -0.0365, -0.0002,  0.0006]])), ('fc1.bias', tensor([-9.6596e-02, -1.5201e-02, -1.8509e-02,  2.9181e-02,  1.1015e-02,
         1.1403e-03, -1.1423e-02, -9.1567e-05, -5.0528e-03, -6.5650e-03,
        -1.8195e-02, -1.4723e-02, -1.3543e-02, -2.2379e-02, -4.9507e-03,
        -7.3408e-05, -1.0404e-02,  3.0550e-03, -1.2495e-01, -3.3072e-02,
         1.4398e-02, -1.0310e-02,  2.6151e-02, -2.4285e-02,  5.8300e-02,
        -1.2565e-01,  2.8762e-03,  1.5452e-02, -1.4524e-02,  1.5347e-02,
        -3.1394e-03, -1.5359e-02, -1.2894e-02, -2.9385e-02,  1.0337e-02,
        -2.6

In [21]:
#validate
validate_loss = 0.0
validate_count = 0
y_pred = []
y_true = []
for i, data in enumerate(validloader, 0):
    best_net.eval()
    inputs, labels = data
    outputs = best_net(inputs.float())
    loss = loss_func(outputs, labels.float())
    for item in outputs:
        y_pred.append(item.item())
    for item in labels:
        y_true.append(item.item())
    validate_loss += loss.item()
    validate_count += 1
print(y_true)
print(y_pred)
print('validation loss: ', validate_loss / validate_count)

[1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0]
[0.6241171956062317, 0.06185154244303703, 1.0, 0.002084453823044896, 1.0, 0.5700456500053406, 1.0, 1.0, 0.9492681622505188, 0.252703994512558, 1.0, 1.0, 1.0, 1.0, 0.89503413438797, 0.02312004566192627, 0.004640539642423391, 0.26173868775367737, 0.4711877405643463, 0.009015600197017193, 1.0, 0.8429558873176575, 0.22323746979236603, 1.0, 1.0, 1.0, 0.33360981941223145, 0.0, 0.04098689556121826, 0.872113049030304, 2.674261668289546e-05, 1.0, 0.7485087513923645, 0.0, 0.22146910429000854, 1.0, 1.

In [24]:
from sklearn.metrics import balanced_accuracy_score, accuracy_score
y_pred_round = []
for pred in y_pred: #keep the network outputs within acceptable bounds
    y_pred_round.append(min(max(round(pred), 0),1))
print(y_pred_round)
print(y_true)
print(balanced_accuracy_score(y_true, y_pred_round))
print(accuracy_score(y_true, y_pred_round))

[1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0,

In [12]:
torch.save(best_net.state_dict(), 'curr_best.pyt')